# Projeto de Criação do relatório de Voluntários CHECK IN 2025

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.Builder().appName("ModelagemVoluntarios").getOrCreate()

25/03/29 21:51:44 WARN Utils: Your hostname, Matheuss-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.135 instead (on interface en0)
25/03/29 21:51:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/29 21:52:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [49]:
nomeArquivo = "Escalas_2025"

escala = spark.read.csv(
    f"./Escalas/{nomeArquivo}.csv",
    sep=";",
    header=True
)

## Criando a base de supervisores

In [ ]:
supers = escala.filter(F.col("Descritivo") == "Supervisor")

supers1 = supers.groupBy("Descritivo", "S1").count()
supers1 = supers1.withColumnRenamed("S1", "Nome")
supers2 = supers.groupBy("Descritivo", "S2").count()
supers2 = supers2.withColumnRenamed("S2", "Nome")
supers3 = supers.groupBy("Descritivo", "S3").count()
supers3 = supers3.withColumnRenamed("S3", "Nome")
supers4 = supers.groupBy("Descritivo", "S4").count()
supers4 = supers4.withColumnRenamed("S4", "Nome")
supers5 = supers.groupBy("Descritivo", "S5").count()
supers5 = supers5.withColumnRenamed("S5", "Nome")

supers = supers1.unionByName(supers2)
supers = supers.unionByName(supers3)
supers = supers.unionByName(supers4)
supers = supers.unionByName(supers5)

supers = supers.drop("count")
supers = supers.orderBy(F.col("Nome").asc())
supers.show()

## Criando a base de Datas de Serviço & Horários

In [ ]:
datasEHorarios = escala.filter(
    (F.col("Descritivo") == "Data") | 
    (F.col("Descritivo") == "Horário")
)

## DATAS
datas = datasEHorarios.filter(F.col("Descritivo") == "Data")
datas1 = datas.groupBy("Descritivo", "S1").count()
datas1 = datas1.withColumnRenamed("S1", "info")
datas2 = datas.groupBy("Descritivo", "S2").count()
datas2 = datas2.withColumnRenamed("S2", "info")
datas3 = datas.groupBy("Descritivo", "S3").count()
datas3 = datas3.withColumnRenamed("S3", "info")
datas4 = datas.groupBy("Descritivo", "S4").count()
datas4 = datas4.withColumnRenamed("S4", "info")
datas5 = datas.groupBy("Descritivo", "S5").count()
datas5 = datas5.withColumnRenamed("S5", "info")

datas = datas1.unionByName(datas2)
datas = datas.unionByName(datas3)
datas = datas.unionByName(datas4)
datas = datas.unionByName(datas5)
datas = datas.drop("count")

## Horas
horas = datasEHorarios.filter(F.col("Descritivo") == "Horário")
horas1 = horas.groupBy("Descritivo", "S1").count()
horas1 = horas1.withColumnRenamed("S1", "info")
horas2 = horas.groupBy("Descritivo", "S2").count()
horas2 = horas2.withColumnRenamed("S2", "info")
horas3 = horas.groupBy("Descritivo", "S3").count()
horas3 = horas3.withColumnRenamed("S3", "info")
horas4 = horas.groupBy("Descritivo", "S4").count()
horas4 = horas4.withColumnRenamed("S4", "info")
horas5 = horas.groupBy("Descritivo", "S5").count()
horas5 = horas5.withColumnRenamed("S5", "info")

horas = horas1.unionByName(horas2)
horas = horas.unionByName(horas3)
horas = horas.unionByName(horas4)
horas = horas.unionByName(horas5)
horas = horas.drop("count").drop_duplicates()


horas.show()


## Relatório Geral

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

escala_S1 = escala.select("Descritivo", "S1")
escala_S1 = escala_S1.withColumn("index", 
                                 F.monotonically_increasing_id())
window_spec = Window.orderBy("index").rowsBetween(
    Window.unboundedPreceding, Window.currentRow
)

escala_S1 = (
    escala_S1.withColumn(
        "Horario_Servico",
        F.when(F.col("S1").like("%h"), F.col("S1")).otherwise(None)
    )
    .withColumn(
        "Data",
        F.when(F.col("S1").like("%-%"), F.col("S1")).otherwise(None)
    )

    .withColumn("Horario_Servico", F.last("Horario_Servico", True).over(window_spec))
    .withColumn("Data", F.last("Data", True).over(window_spec))
    .drop("index")  
)

escala_S1 = escala_S1.filter(
    (F.col("Descritivo") != "Data") &
    (F.col("Descritivo") != "Horario") 
).withColumnRenamed("S1", "Voluntario")


In [ ]:
escala_S2 = escala.select("Descritivo", "S2")
escala_S2 = escala_S2.withColumn("index", 
                                 F.monotonically_increasing_id())
window_spec = Window.orderBy("index").rowsBetween(
    Window.unboundedPreceding, Window.currentRow
)

escala_S2 = (
    escala_S2.withColumn(
        "Horario_Servico",
        F.when(F.col("S2").like("%h"), F.col("S2")).otherwise(None)
    )
    .withColumn(
        "Data",
        F.when(F.col("S2").like("%-%"), F.col("S2")).otherwise(None)
    )

    .withColumn("Horario_Servico", F.last("Horario_Servico", True).over(window_spec))
    .withColumn("Data", F.last("Data", True).over(window_spec))
    .drop("index")  
)

escala_S2 = escala_S2.filter(
    (F.col("Descritivo") != "Data") &
    (F.col("Descritivo") != "Horario") 
).withColumnRenamed("S2", "Voluntario")


In [ ]:
escala_S3 = escala.select("Descritivo", "S3")
escala_S3 = escala_S3.withColumn("index", 
                                 F.monotonically_increasing_id())
window_spec = Window.orderBy("index").rowsBetween(
    Window.unboundedPreceding, Window.currentRow
)

escala_S3 = (
    escala_S3.withColumn(
        "Horario_Servico",
        F.when(F.col("S3").like("%h"), F.col("S3")).otherwise(None)
    )
    .withColumn(
        "Data",
        F.when(F.col("S3").like("%-%"), F.col("S3")).otherwise(None)
    )

    .withColumn("Horario_Servico", F.last("Horario_Servico", True).over(window_spec))
    .withColumn("Data", F.last("Data", True).over(window_spec))
    .drop("index")  
)

escala_S3 = escala_S3.filter(
    (F.col("Descritivo") != "Data") &
    (F.col("Descritivo") != "Horario") 
).withColumnRenamed("S3", "Voluntario")


In [ ]:
escala_S4 = escala.select("Descritivo", "S4")

# Para casos com mais pessoas
escala_S4 = escala_S4.withColumn("Descritivo", 
        F.when(
            F.col("Descritivo").isNull(), F.lit("Voluntario")
            ).otherwise(F.col("Descritivo"))
        )

escala_S4 = escala_S4.withColumn("index", 
                                 F.monotonically_increasing_id())
window_spec = Window.orderBy("index").rowsBetween(
    Window.unboundedPreceding, Window.currentRow
)

escala_S4 = (
    escala_S4.withColumn(
        "Horario_Servico",
        F.when(F.col("S4").like("%h"), F.col("S4")).otherwise(None)
    )
    .withColumn(
        "Data",
        F.when(F.col("S4").like("%-%"), F.col("S4")).otherwise(None)
    )
    .withColumn("Horario_Servico", F.last("Horario_Servico", True).over(window_spec))
    .withColumn("Data", F.last("Data", True).over(window_spec))
    .drop("index")  
)

escala_S4 = escala_S4.filter(
    (F.col("Descritivo") != "Data") &
    (F.col("Descritivo") != "Horario")
).withColumnRenamed("S4", "Voluntario")


In [ ]:
escala_S5 = escala.select("Descritivo", "S5")
# Para casos com mais pessoas
escala_S5 = escala_S5.withColumn("Descritivo", 
        F.when(
            F.col("Descritivo").isNull(), F.lit("Voluntario")
            ).otherwise(F.col("Descritivo"))
        )
escala_S5 = escala_S5.withColumn("index", 
                                 F.monotonically_increasing_id())
window_spec = Window.orderBy("index").rowsBetween(
    Window.unboundedPreceding, Window.currentRow
)

escala_S5 = (
    escala_S5.withColumn(
        "Horario_Servico",
        F.when(F.col("S5").like("%h"), F.col("S5")).otherwise(None)
    )
    .withColumn(
        "Data",
        F.when(F.col("S5").like("%-%"), F.col("S5")).otherwise(None)
    )

    .withColumn("Horario_Servico", F.last("Horario_Servico", True).over(window_spec))
    .withColumn("Data", F.last("Data", True).over(window_spec))
    .drop("index")  
)

escala_S5 = escala_S5.filter(
    (F.col("Descritivo") != "Data") &
    (F.col("Descritivo") != "Horario")
).withColumnRenamed("S5", "Voluntario")


In [55]:
relatorio = escala_S1.unionByName(escala_S2)
relatorio = relatorio.unionByName(escala_S3)
relatorio = relatorio.unionByName(escala_S4)
relatorio = relatorio.unionByName(escala_S5)

## Convertendo arquivo para .xlsx

In [ ]:
dados = relatorio.collect()

dados_dict = [row.asDict() for row in dados]
dados_dict

25/03/29 22:26:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/29 22:26:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/29 22:26:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/29 22:26:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/29 22:26:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/29 22:26:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/29 2

[{'Descritivo': 'Supervisor',
  'Voluntários': 'Gabriela Andrade',
  'Horario_Servico': '09/11h',
  'Data': '02-Mar'},
 {'Descritivo': 'Voluntário',
  'Voluntários': 'Diogo Rocha',
  'Horario_Servico': '09/11h',
  'Data': '02-Mar'},
 {'Descritivo': 'Voluntário',
  'Voluntários': 'Karen Santana',
  'Horario_Servico': '09/11h',
  'Data': '02-Mar'},
 {'Descritivo': 'Voluntário',
  'Voluntários': 'Rebecca',
  'Horario_Servico': '09/11h',
  'Data': '02-Mar'},
 {'Descritivo': 'Voluntário',
  'Voluntários': 'Bruno Santana',
  'Horario_Servico': '09/11h',
  'Data': '02-Mar'},
 {'Descritivo': 'Voluntário',
  'Voluntários': 'Gabriela Silva',
  'Horario_Servico': '09/11h',
  'Data': '02-Mar'},
 {'Descritivo': 'Supervisor',
  'Voluntários': 'Alanis Rodrigues',
  'Horario_Servico': '18h',
  'Data': '02-Mar'},
 {'Descritivo': 'Voluntário',
  'Voluntários': 'Graciele Lacerda',
  'Horario_Servico': '18h',
  'Data': '02-Mar'},
 {'Descritivo': 'Voluntário',
  'Voluntários': 'Thamires Lima',
  'Horario_S

In [63]:
import pandas as pd
df_pandas = pd.DataFrame(dados_dict)
df_pandas.to_csv('./Escalas/Relatorio_2025.csv', index=False)